In [41]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
import numpy as np
import pandas as pd
import random


In [42]:
data = pd.read_csv('drive/My Drive/all_haiku.csv')


data['haiku'] = data['0']+' '+data['1']+' '+data['2']
data['haiku'] = data['haiku'].apply(lambda x: str(x).strip().replace('-','').lower()+' stopgenerating')

tokenizer = Tokenizer()

corpus = data.iloc[0:5000,-1].to_list()

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

print(total_words)
print(tokenizer.word_index)

7059
{'stopgenerating': 1, 'the': 2, 'a': 3, 'of': 4, 'in': 5, 'on': 6, 'my': 7, 'to': 8, 'rain': 9, 'moon': 10, 'her': 11, 'and': 12, 'night': 13, 'i': 14, 'day': 15, 'morning': 16, 'from': 17, 'at': 18, 'winter': 19, 'with': 20, 'summer': 21, 'for': 22, 'autumn': 23, 'an': 24, 'spring': 25, 'old': 26, 'first': 27, 'his': 28, 'snow': 29, 'new': 30, 'she': 31, 'cold': 32, 'wind': 33, 'light': 34, 'sky': 35, 'leaves': 36, 'last': 37, 'one': 38, 'after': 39, 'out': 40, 'by': 41, 'fog': 42, 'me': 43, 'another': 44, 'blue': 45, 'white': 46, 'through': 47, 'window': 48, 'all': 49, 'between': 50, 'scent': 51, 'into': 52, 'end': 53, 'red': 54, 'clouds': 55, 'this': 56, 'is': 57, 'silence': 58, 'full': 59, 'still': 60, 'no': 61, 'coffee': 62, 'long': 63, 'sun': 64, 'sound': 65, 'tea': 66, 'stars': 67, 'over': 68, 'evening': 69, 'up': 70, 'smell': 71, 'train': 72, 'we': 73, 'storm': 74, 'home': 75, 'our': 76, 'late': 77, 'empty': 78, 'tree': 79, 'afternoon': 80, 'as': 81, 'lights': 82, 'two': 8

In [43]:
input_sequences = []

for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 50, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(xs, ys, epochs=10, verbose=1)

print(model)
print(model.summary())

Epoch 1/10
1406/1406 [==============================] - 109s 78ms/step - loss: 6.5362 - accuracy: 0.1704
Epoch 2/10
1406/1406 [==============================] - 111s 79ms/step - loss: 5.9635 - accuracy: 0.2037
Epoch 3/10
1406/1406 [==============================] - 107s 76ms/step - loss: 5.5794 - accuracy: 0.2179
Epoch 4/10
1406/1406 [==============================] - 106s 75ms/step - loss: 5.2890 - accuracy: 0.2284
Epoch 5/10
1406/1406 [==============================] - 109s 78ms/step - loss: 5.0143 - accuracy: 0.2384
Epoch 6/10
1406/1406 [==============================] - 111s 79ms/step - loss: 4.7461 - accuracy: 0.2450
Epoch 7/10
1406/1406 [==============================] - 116s 82ms/step - loss: 4.4837 - accuracy: 0.2556
Epoch 8/10
 277/1406 [====>.........................] - ETA: 1:27 - loss: 4.0812 - accuracy: 0.2758

In [ ]:
input_sequences = []

for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [ ]:
seed_text = random.choice(list(tokenizer.word_index.keys()))
next_words = 50
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
      if index == predicted:
        output_word = word
        break
    if output_word == 'stopgenerating':
      break
    else:
      seed_text += " " + output_word
    
print(seed_text)